In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## Drawer

In [4]:
from pkg.utils.code_scraps import add_drawer
drawer_bot, drawer, handle = add_drawer(pscene, center=(1,0,0.5))

### prepare planner

In [5]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [6]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [7]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

   Use a production WSGI server instead.
 * Debug mode: off
['drawer', 'handle']
(0, 'drawer_bot')


In [8]:
goal_nodes = [(2,'drawer_bot')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 36/36 agents
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawe

try: 2 - (1, 'grip0')->(2, 'grip0')
branching: 3->14 (0.57/30.0 s, steps/err: 1(6.3750743866 ms)/0)
try transition motion
result: 0 - (0, 'drawer_bot')->(0, 'grip0') = success
try: 13 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 2 - (1, 'grip0')->(2, 'grip0')
collision
collision
try constrained motion
try constrained motion
transition motion tried: True
constrained motion tried: False
branching: 0->15 (0.6/30.0 s, steps/err: 35(211.160182953 ms)/0.00166063333032)
constrained motion tried: False
try constrained motion
Motion Plan Failure
result: 0 - (0, 'drawer_bot')->(0, 'grip0') = success
result: 5 - (1, 'grip0')->(2, 'grip0') = fail
Motion Plan Failure
try: 5 - (1, 'grip0')->(2, 'grip0')
collision
result: 5 - (1, 'grip0')->(2, 'grip0') = fail
constrained motion tried: False
branching: 0->16 (0.62/30.0 s, steps/err: 83(405.232191086 ms)/0.00147877888578)
try: 15 - (0, 'grip0')->(1, 'grip0')
transition motion tried: True
transition motion tried: True
try constrained motion
Motion Plan F

branching: 27->32 (0.85/30.0 s, steps/err: 1(7.95698165894 ms)/0)
result: 2 - (1, 'grip0')->(2, 'grip0') = success
end
branching: 2->33 (0.85/30.0 s, steps/err: 21(270.570039749 ms)/0.000178475446113)
constrained motion tried: True
result: 13 - (0, 'drawer_bot')->(0, 'grip0') = success
++ adding return motion to acquired answer ++
branching: 13->34 (0.86/30.0 s, steps/err: 47(147.061109543 ms)/0.00102557172678)
try: 33 - (2, 'grip0')->(2, 'drawer_bot')
result: 2 - (1, 'grip0')->(2, 'grip0') = success
try transition motion
branching: 2->35 (0.88/30.0 s, steps/err: 21(318.061113358 ms)/0.000178475446113)
try: 35 - (2, 'grip0')->(2, 'drawer_bot')
try: 21 - (1, 'grip0')->(2, 'grip0')
transition motion tried: True
Goal reached
try constrained motion
result: 35 - (2, 'grip0')->(2, 'drawer_bot') = success
Goal reached
branching: 35->36 (0.91/30.0 s, steps/err: 1(6.65593147278 ms)/0)
result: 8 - (0, 'drawer_bot')->(0, 'grip0') = success
++ adding return motion to acquired answer ++
branching: 

======================= terminated 25: Stop called from other agent  (1.1/30.0) ===============================
======================= terminated 19: Stop called from other agent  (1.2/30.0) ===============================
branching: 21->49 (1.16/30.0 s, steps/err: 21(255.892038345 ms)/0.000184081431491)
result: 23 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
======================= terminated 29: Stop called from other agent  (1.2/30.0) ===============================
======================= terminated 30: Stop called from other agent  (1.2/30.0) ===============================
======================= terminated 21: Stop called from other agent  (1.2/30.0) ===============================
======================= terminated 32: Stop called from other agent  (1.2/30.0) ===============================
======================= terminated 27: Stop called from other agent  (1.2/30.0) ===============================
======================= terminated 9: Stop called from other agent  (1.2/30.

======================= terminated 13: required answers acquired  (1.4/30.0) ===============================
transition motion tried: False
transition motion tried: False
Motion Plan Failure
Motion Plan Failure
result: 40 - (2, 'grip0')->(2, 'drawer_bot') = fail
result: 39 - (2, 'grip0')->(2, 'drawer_bot') = fail
======================= terminated 15: Stop called from other agent  (1.5/30.0) ===============================
======================= terminated 11: Stop called from other agent  (1.5/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 4 - (0, 'grip0')->(0, 'drawer_bot') = fail
======================= terminated 16: Stop called from other agent  (1.6/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 11 - (0, 'grip0')->(0, 'drawer_bot') = fail
transition motion tried: False
Motion Plan Failure
======================= terminated 17: Stop called from other agent  (1.7/30.0) =============

In [9]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(0, 'drawer_bot')->(0, 'grip0')
(0, 'grip0')->(1, 'grip0')
(1, 'grip0')->(2, 'grip0')
(2, 'grip0')->(2, 'drawer_bot')
(2, 'drawer_bot')->(2, 'drawer_bot')


### push back

In [10]:
gscene.show_pose(crob.home_pose)
term_state = snode_schedule[-1].state
print(pscene.subject_name_list)
print(term_state.node)

['drawer', 'handle']
(2, 'drawer_bot')


In [11]:
goal_nodes = [(1,'drawer_bot')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 36/36 agents
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'grip0')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try transition motion
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (2, 'drawer_bot')->(2, 'grip0')
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try transition motion
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (2, 'drawer_bot')->(2, 'drawer_b

try: 11 - (1, 'grip0')->(1, 'drawer_bot')
try transition motion
try: 1 - (2, 'grip0')->(2, 'drawer_bot')
try transition motion
result: 3 - (2, 'grip0')->(1, 'grip0') = success
branching: 3->13 (0.9/30.0 s, steps/err: 21(270.457029343 ms)/0.000769397486448)
try transition motion
try: 4 - (2, 'grip0')->(2, 'grip0')
try: 6 - (2, 'drawer_bot')->(2, 'grip0')
try: 6 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 13 - (1, 'grip0')->(2, 'grip0')
try: 3 - (2, 'grip0')->(2, 'drawer_bot')
try: 13 - (1, 'grip0')->(1, 'drawer_bot')
end
try transition motion
try transition motion
try constrained motion
Goal reached
constrained motion tried: True
Goal reached
try constrained motion
transition motion tried: False
result: 11 - (1, 'grip0')->(1, 'drawer_bot') = success
try: 2 - (2, 'drawer_bot')->(2, 'grip0')
result: 6 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
end
result: 13 - (1, 'grip0')->(1, 'drawer_bot') = success
Motion Plan Failure
transition motion tried: False
try: 16 - (1, 'grip0')->(1, 'drawe

======================= terminated 10: Stop called from other agent  (1.4/30.0) ===============================
branching: 10->33 (1.36/30.0 s, steps/err: 21(313.915014267 ms)/0.000534798646952)
======================= terminated 2: Stop called from other agent  (1.4/30.0) ===============================
branching: 7->34 (1.37/30.0 s, steps/err: 21(467.945098877 ms)/0.0001780200844)
======================= terminated 11: Stop called from other agent  (1.4/30.0) ===============================
======================= terminated 12: Stop called from other agent  (1.4/30.0) ===============================
======================= terminated 27: Stop called from other agent  (1.4/30.0) ===============================
transition motion tried: False
transition motion tried: False
joint min
Motion Plan Failure
transition motion tried: False
======================= terminated 0: Stop called from other agent  (1.4/30.0) ===============================
Motion Plan Failure
======================= 

======================= terminated 6: Stop called from other agent  (1.6/30.0) ===============================
======================= terminated 21: Stop called from other agent  (1.6/30.0) ===============================
======================= terminated 28: Stop called from other agent  (1.6/30.0) ===============================
======================= terminated 19: Stop called from other agent  (1.6/30.0) ===============================
transition motion tried: False
Motion Plan Failure
transition motion tried: False
Motion Plan Failure
transition motion tried: False
Motion Plan Failure
result: 7 - (1, 'grip0')->(1, 'drawer_bot') = fail
result: 18 - (2, 'grip0')->(2, 'drawer_bot') = fail
result: 4 - (2, 'grip0')->(2, 'drawer_bot') = fail
======================= terminated 1: Stop called from other agent  (1.7/30.0) ===============================
======================= terminated 25: Stop called from other agent  (1.7/30.0) ===============================
=======================

In [12]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(2, 'drawer_bot')->(2, 'grip0')
(2, 'grip0')->(1, 'grip0')
(1, 'grip0')->(1, 'drawer_bot')
(1, 'drawer_bot')->(1, 'drawer_bot')
